### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    B2E0B38C3885C4F413E0E2C96AA59951 DB61E81D580E9097AD568BDB1031019C
```    
where:
```
    B2E0B38C3885C4F413E0E2C96AA59951 is a ciphertext
    
    DB61E81D580E9097AD568BDB1031019C is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace is a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of SCA{XXXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [13]:
import numpy as np
import binascii
import string
import random
import numpy.matlib
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4002

import sca_training
#----------------------------------------------------------------------------
# This function calls tested binary ../wbcenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32,324}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')
            output = s.recv(1024)

    #Get the ciphertext (first 32 symbols) and the trace (last 32 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace

#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [14]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)

Binary output: b'3F7BA33CBAF53C4BBE5E849C3456E6DA 59E733380BA3D9CFEB81F5718344AA4A4703DC3E3C69771278A06D5A12F113A12046186128F2391CB26EC43EB277BE387AD9ACED4B6208C342AC5C8AA6703505EC4D671E7ED603030303060302050304030403030305DB455740D03CE64478DC911E034F'
Ciphertext as numpy array: [ 63 123 163  60 186 245  60  75 190  94 132 156  52  86 230 218]
Trace as numpy array: [ 89 231  51  56  11 163 217 207 235 129 245 113 131  68 170  74  71   3
 220  62  60 105 119  18 120 160 109  90  18 241  19 161  32  70  24  97
  40 242  57  28 178 110 196  62 178 119 190  56 122 217 172 237  75  98
   8 195  66 172  92 138 166 112  53   5 236  77 103  30 126 214   3   3
   3   3   6   3   2   5   3   4   3   4   3   3   3   5 219  69  87  64
 208  60 230  68 120 220 145  30   3  79]


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [15]:
#This is a first solution that uses ShiftRows operation over the traces.

#The principal difference in comparison with the previous task is that the leakage is no longer a state byte, but a Hamming weight of this byte.
#Therefore, different keys (correct and wrong) can result in the same Hamming weight for one encryption.

#To mitigate this tiny problem several encryptions (more than 6) shall be used. 
num_enc = 10

traces = np.zeros((num_enc, 100), dtype=np.uint8)
ctexts = np.zeros((num_enc, 16), dtype=np.uint8)

for iEnc in range(num_enc):
    ptext = '%032x' % random.randrange(16**32)
    output, ctext, trace = binary_aes128_encrypt(ptext, verbose=False)
    ctexts[iEnc,:] = ctext
    traces[iEnc,:] = trace
    
#Define numpy structure to keep the recovered key
key = np.zeros((16), dtype=np.uint8)

#Predefine all key candidates as a matrix num_enc x 256 
key_cand = np.matlib.repmat(np.arange(256).astype(np.uint8), num_enc,1)

#Shift the leakage samples so that they are aligned with the ciphertext bytes
shifted_trace = sca_training.shift_rows(traces)

#Define numpy structure to keep the recovered key
key = np.zeros((16), dtype=np.uint8)

#Predefine all key candidates as a matrix num_enc x 256 
key_cand = np.matlib.repmat(np.arange(256).astype(np.uint8), num_enc,1)

for iByte in range(16):
    
    #Xor all the key candidates with the ciphertext byte: k ^ c[i,j]
    #k is a matrix of 'num_enc x 256' elements (key candidate from 0 to 255 for each encryption)
    #i is the iterator from 0 to the number of encryptions num_enc, numpy can xor matrix with vector
    #j is iByte
    sbox_out = np.bitwise_xor(key_cand, ctexts[:,iByte].reshape(num_enc,1))

    #Apply inverse Sbox: invSbox[k ^ c[i, j]]
    sbox_in  = sca_training.invSbox[sbox_out]
    
    #Compute the Hamming weight (i.e., a model of the leakage): HW(invSbox[k ^ c[i, j]])
    hw_sbox_in = sca_training.HW_uint8[sbox_in]

    #Since we don't know the position of the leakage for this byte we need to iterate over all available leakage moments and compute the number of hits
    #between the "real" and modelled leakages between for all of them.
    #The number of hits for the correct key shall be equal to the num_enc when the byte index 'iByte' leaks in the trace
    #If 'iSample' and 'iByte' don't correspond (i.e. byte is not leakage at this moment) or the key byte is wrong then 
    #the number of hits shall be less than num_enc
    #If num_enc is relatively small (i.e., less than 5) then for some wrong key candidates the number of hits might be still equal to num_enc.
    #This can be mitigated with increase of num_enc. Typically num_enc > 6 shall be enough to find the key
    
    for iSample in range(traces.shape[1]):
        best_keys = np.where(np.sum(hw_sbox_in==traces[:,iSample].reshape(num_enc,1), axis=0) == num_enc)[0]

        if len(best_keys) == 0:
            print('No key candidates remained for ciphertext byte', iByte, 'and sample index', iSample)

        if len(best_keys) == 1:
            print('Only one key candidate remained for ciphertext byte', iByte, 'and sample index', iSample)
            key[iByte] = best_keys[0]

        if len(best_keys) > 1:
            print('Several key candidates remained for ciphertext byte', iByte, 'and sample index', iSample)
            print(best_keys)
            
    print('\n')

#Get master key using the provided binary
key_schedule = sca_training.inverse_key_expansion(key)
print(key_schedule.shape)

print('Master key in hex:', key_schedule[0,0,:])

print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))

No key candidates remained for ciphertext byte 0 and sample index 0
No key candidates remained for ciphertext byte 0 and sample index 1
No key candidates remained for ciphertext byte 0 and sample index 2
No key candidates remained for ciphertext byte 0 and sample index 3
No key candidates remained for ciphertext byte 0 and sample index 4
No key candidates remained for ciphertext byte 0 and sample index 5
No key candidates remained for ciphertext byte 0 and sample index 6
No key candidates remained for ciphertext byte 0 and sample index 7
No key candidates remained for ciphertext byte 0 and sample index 8
No key candidates remained for ciphertext byte 0 and sample index 9
No key candidates remained for ciphertext byte 0 and sample index 10
No key candidates remained for ciphertext byte 0 and sample index 11
No key candidates remained for ciphertext byte 0 and sample index 12
No key candidates remained for ciphertext byte 0 and sample index 13
No key candidates remained for ciphertext by

In [16]:
#Encrypt given input to check the correctness
from Crypto.Cipher import AES

aes = AES.new(b'HEIG{Catchmeiyc}', AES.MODE_ECB)

ctext = aes.encrypt(binascii.unhexlify('010203040506070809000a0b0c0d0e1f'))

print(binascii.hexlify(ctext))

b'3f7ba33cbaf53c4bbe5e849c3456e6da'
